In [25]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.base import BaseEstimator,clone

import pandas as pd
import numpy as np


In [26]:
dat_tmp = pd.read_csv("../data/student-mat.csv",sep=';',)

In [27]:
dat = pd.DataFrame(dat_tmp['school'].str.replace('"','').str.split(";").to_list(),columns=list(dat_tmp.columns))

In [45]:
dat

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,course,other,1,2,2,no,yes,yes,no,yes,yes,no,no,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,course,other,1,1,3,no,no,no,no,no,yes,no,no,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother,3,1,0,no,no,no,no,no,yes,yes,no,4,4,1,3,4,5,0,11,12,10


In [28]:
pd.set_option("display.max_columns",50)

In [63]:
target = 'G3'


In [64]:
X1raw = dat[['studytime','sex','school','Medu']].copy()

In [211]:
X1processed = X1raw[['studytime','Medu']].copy()
X1processed['sex_F'] = np.where(X1raw['sex']=='F',1,0)
X1processed['school_GP'] = np.where(X1raw['school']=='GP',1,0)

X1processed['int']=1
X1processed=X1processed.astype(float)
Y = dat[target].astype(float)

In [212]:

X_train,X_test,y_train,y_test = train_test_split(X1processed,Y,test_size=.2,random_state=42,shuffle=True)

In [213]:
level1_model = LinearRegression(fit_intercept=False).fit(X_train,y_train)
y_fit,y_pred = level1_model.predict(X_train),level1_model.predict(X_test)

In [214]:
r2_score(y_test,y_pred)

0.026255948733968637

In [215]:
pd.DataFrame({'feat':X_train.columns,'coef':level1_model.coef_})

,feat,coef
0,studytime,0.901257
1,Medu,0.845745
2,sex_F,-1.028995
3,school_GP,0.055855
4,int,6.671966


In [264]:

lr = .000001

In [265]:


coefs1 = np.random.normal(0,.1,4)
coefs2_1 = np.random.normal(0,.1,2)

In [266]:
def model(X,coefs1,coefs2_1):
    
    yhat = (
        X[['sex_F','school_GP','Medu','int']]@coefs1
        +
        X['studytime']*(X[['Medu','int']]@coefs2_1)
    )

    return yhat


In [267]:
for e in range(100000):
    yhat = model(X_train,coefs1,coefs2_1)

    err = (yhat - y_train)

    grad_coefs1 = err@X_train[['sex_F','school_GP','Medu','int']]
    grad_coefs2_1 = (err*X_train['studytime'])@X_train[['Medu','int']]

    coefs1 -= lr*grad_coefs1
    coefs2_1 -= lr*grad_coefs2_1

In [268]:
print(grad_coefs1,grad_coefs2_1
      )

sex_F         0.468918
school_GP     3.779183
Medu          2.760655
int         -12.189693
dtype: float64 Medu   -1.249902
int     3.838885
dtype: float64


In [269]:
y_pred = model(X_test,coefs1,coefs2_1)

In [270]:
r2_score(y_test,y_pred)

0.02399115449518363

In [271]:
coefs2_1

Medu   -0.424480
int     2.188292
dtype: float64

In [272]:
print(coefs1,coefs2_1
      )

sex_F       -1.036907
school_GP    0.330221
Medu         1.749174
int          3.685278
dtype: float64 Medu   -0.424480
int     2.188292
dtype: float64
